# Profile Correction

This script creates a radial correction map to account for systematic differences between measured and modeled off-axis profiles.

**What you need:**
- Large field (40×40 cm recommended) crossline/inline profiles at 10 cm depth
- ASCII format from water tank scanning system

**What to do:**
1. Set `PROFILE_MEASUREMENTS` to your profile data (cross-profile) taken at `PROFILE_RADIAL_MEASUREMENT_POINTS` distances.
2. Set `FIELD_SIZE` to match your measurement (typically 400 mm)
3. Adjust `COORD_MAP` if needed for your coordinate system
4. Run and verify the corrected profile matches measurements

**Output:**
- `profile_corrections` array [[distances], [correction_ratios]]


In [ ]:
from sympy import false
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import pydicom
from pydose_rt import DoseEngine
from pydose_rt.data import MachineConfig, Phantom, loaders, Beam
from pydose_rt.utils.utils import sample_tensor_nearest
from pydose_rt.physics.fluence.fluence_modeling import compute_profile_ratios, create_radial_correction_map

In [ ]:
measurements = loaders.load_asc_measurements("/home/bolo/Documents/PyDoseRT/test_data/10 MV Photons/TrueBeam X10 Squares OK.asc", coord_map=("X", "Z", "Y"))
measurement = [measurement for measurement in measurements if measurement["header_dict"]["FSZ"] == ['300', '300']][1]


In [ ]:

diffs = np.diff(measurement['coords_engine'], axis=0)
seg_len = np.linalg.norm(diffs, axis=1)
dist = np.concatenate(([0.0], np.cumsum(seg_len)))
ticks = dist - np.mean(dist)

In [ ]:
plt.plot(ticks, measurement['dose'])

In [ ]:
CALIBRATION_MU = 110
FIELD_SIZE = 400  # mm

PROFILE_RADIAL_MEASUREMENT_POINTS = np.linspace(0, 300, 301)
PROFILE_MEASUREMENTS = [100.   , 100.   , 100.   , 100.   , 100.   , 100.039, 100.2  ,
       100.221, 100.28 , 100.209, 100.3  , 100.3  , 100.3  , 100.312,
       100.4  , 100.5  , 100.5  , 100.506, 100.7  , 100.705, 100.806,
       100.907, 100.993, 100.9  , 100.9  , 100.9  , 100.997, 101.098,
       101.199, 101.2  , 101.301, 101.389, 101.311, 101.39 , 101.491,
       101.5  , 101.5  , 101.594, 101.6  , 101.6  , 101.681, 101.782,
       101.8  , 101.884, 101.985, 102.   , 102.   , 102.   , 102.075,
       102.176, 102.123, 102.178, 102.121, 102.18 , 102.281, 102.3  ,
       102.3  , 102.438, 102.43 , 102.329, 102.363, 102.4  , 102.4  ,
       102.466, 102.5  , 102.5  , 102.436, 102.4  , 102.458, 102.5  ,
       102.5  , 102.5  , 102.444, 102.457, 102.558, 102.541, 102.5  ,
       102.554, 102.6  , 102.549, 102.452, 102.449, 102.549, 102.503,
       102.4  , 102.45 , 102.545, 102.556, 102.454, 102.44 , 102.5  ,
       102.5  , 102.5  , 102.5  , 102.5  , 102.455, 102.44 , 102.5  ,
       102.461, 102.443, 102.5  , 102.5  , 102.465, 102.4  , 102.4  ,
       102.437, 102.468, 102.43 , 102.474, 102.371, 102.332, 102.4  ,
       102.377, 102.329, 102.37 , 102.3  , 102.3  , 102.324, 102.362,
       102.2  , 102.2  , 102.2  , 102.178, 102.1  , 102.118, 102.2  ,
       102.177, 102.1  , 102.082, 102.   , 102.   , 102.   , 102.   ,
       101.985, 101.9  , 101.884, 101.8  , 101.8  , 101.804, 101.9  ,
       101.906, 101.987, 101.792, 101.709, 101.8  , 101.8  , 101.809,
       101.897, 101.803, 101.8  , 101.8  , 101.702, 101.799, 101.701,
       101.7  , 101.6  , 101.689, 101.612, 101.6  , 101.684, 101.61 ,
       101.6  , 101.6  , 101.6  , 101.6  , 101.6  , 101.6  , 101.6  ,
       101.6  , 101.516, 101.5  , 101.5  , 101.417, 101.324, 101.3  ,
       101.3  , 101.3  , 101.169, 101.1  , 101.037, 100.932, 100.834,
       100.735, 100.639, 100.537, 100.5  , 100.432, 100.272, 100.077,
       100.   ,  99.879,  99.689,  99.543,  99.385,  99.241,  99.034,
        98.851,  98.754,  98.561,  98.353,  98.155,  97.901,  97.749,
        97.565,  97.286,  97.059,  96.915,  96.682,  96.425,  96.228,
        96.062,  95.897,  95.603,  95.372,  95.214,  94.942,  94.742,
        94.48 ,  94.169,  94.011,  93.729,  93.462,  93.221,  92.837,
        92.534,  92.232,  91.886,  91.428,  91.05 ,  90.487,  90.024,
        89.405,  88.704,  87.969,  87.084,  86.268,  85.343,  84.316,
        83.219,  82.197,  81.246,  80.04 ,  78.929,  77.619,  76.096,
        74.131,  71.457,  68.186,  63.249,  58.476,  52.365,  46.105,
        39.805,  34.194,  29.579,  25.626,  22.791,  20.255,  18.516,
        16.891,  15.631,  14.779,  13.978,  13.449,  12.922,  12.466,
        12.082,  11.763,  11.46 ,  11.157,  10.854,  10.569,  10.349,
        10.158,   9.876,   9.728,   9.488,   9.257,   9.055,   8.932,
         8.699,   8.473,   8.339,   8.175,   7.973,   7.835,   7.669,
         7.538,   7.388,   7.243,   7.079,   6.939,   6.834,   6.686,
         6.548,   6.444,   6.343,   6.185,   6.046,   5.949,   5.844]

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

resolution = (1.0, 1.0, 1.0)
ct_array_shape = (600, 200, 600)
phantom = Phantom.from_uniform_water(shape=ct_array_shape, spacing=resolution).to(device).to(dtype)
number_of_beams = 1
starting_angle = 0
iso_center = (300.0, 100.0, 300.0)
kernel_size = 601
beam = Beam.create(
    gantry_angle_deg=0.0, 
    number_of_leaf_pairs=60, 
    collimator_angle_deg=0.0, 
    field_size_mm=(FIELD_SIZE, FIELD_SIZE),
    iso_center=iso_center, 
    device=device, 
    dtype=dtype)

machine_config = MachineConfig(
    preset="../../src/pydose_rt/data/machine_presets/umea_10MV.json",
    profile_corrections=None,
    output_factors=None
    )
dose_engine = DoseEngine(
    machine_config, 
    kernel_size,
    dose_grid_spacing=phantom.resolution,
    dose_grid_shape=phantom.density_image.shape,
    beam_template=beam,
    device=device,
    dtype=dtype,
    adjust_values=False
)
dose_engine.calibrate(110)

dose = dose_engine.compute_dose(
    beam,
    density_image=phantom.density_image).detach()

data = []

samples = dose.cpu().detach().numpy()[0, 299:, 100, 300]
samples /= samples[0] / 100

plt.plot(PROFILE_RADIAL_MEASUREMENT_POINTS, samples, label="model")
plt.plot(PROFILE_RADIAL_MEASUREMENT_POINTS, PROFILE_MEASUREMENTS, label="measurement")
plt.legend()
plt.show()

In [ ]:
diag = np.diagonal(dose[0, :, 100].cpu().detach().numpy())
diag = diag[diag.shape[0] // 2 :]
plt.plot(diag)

In [ ]:
# Stage 1: Compute sampled ratios
num_sampling_points = len(PROFILE_RADIAL_MEASUREMENT_POINTS)
sample_points = PROFILE_RADIAL_MEASUREMENT_POINTS  # [0, 10, 20, ..., 500]
distances, ratios = compute_profile_ratios(
    PROFILE_MEASUREMENTS, samples, PROFILE_RADIAL_MEASUREMENT_POINTS, PROFILE_RADIAL_MEASUREMENT_POINTS
)

print(f"Sampled {len(ratios)} points")
print(f"Sample distances: {distances[:5]}... {distances[-5:]}")
print(f"Sample ratios: {ratios[:5]}... {ratios[-5:]}")

# Stage 2: Create correction map
image_shape = (num_sampling_points*2, num_sampling_points*2)  # height, width
pixel_size = 1.0  # 1 mm per pixel
    
correction_map = create_radial_correction_map(
    distances,
    ratios,
    image_shape,
    pixel_size,
).cpu().detach().numpy()

plt.imshow(correction_map)
plt.show()

plt.plot(PROFILE_RADIAL_MEASUREMENT_POINTS, PROFILE_MEASUREMENTS, label="Measured")
plt.plot(PROFILE_RADIAL_MEASUREMENT_POINTS, samples, label=f"Modelled {np.mean(np.abs(PROFILE_MEASUREMENTS - samples))}")
plt.plot(PROFILE_RADIAL_MEASUREMENT_POINTS, correction_map[num_sampling_points:, num_sampling_points] * samples, label=f"Corrected {np.mean(np.abs(PROFILE_MEASUREMENTS - correction_map[num_sampling_points:, num_sampling_points] * samples))}")
plt.legend(loc="lower left")
plt.show()

profiles = np.round(np.stack([distances, ratios], 0), 4)

def fmt(x):
    s = np.format_float_positional(x, trim='-')
    return s if '.' in s else s + '.0'

formatted = np.array2string(
    profiles,
    formatter={'float_kind': fmt},
    separator=', '       # ← comma between values
)

print(formatted)
